# pandas 入门

pandas 主要功能: 

* 具备按轴自动或显示数据对齐功能的数据结构.
* 集成时间序列功能.
* 即能处理时间序列数据也能处理非时间序列数据的数据结构.
* 数学运算和约简(比如对某个轴求和) 可以根据不同的元数据执行.
* 灵活处理缺失数据.
* 合并及其他出现在常见数据库中的关系型运算.

In [13]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

## Series

Series 是一种类似于一维数组的对象，它由一组数据(各种NumPy数据类型)以及一组与之相关的数据标签(即索引)组成。仅由一组数据即可产生最简单的 Series :

In [2]:
obj = Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

Series 的字符串表现形式为: 索引在左边，值在右边。由于我们没有为数据指定索引，于是会自动创建一个 0 到 N1(N 为数据的长度) 的整数型索引, 可以通过 Series 的 values 和 index 属性获取其数组表示形式和索引对象:

In [3]:
obj.values

array([ 4,  7, -5,  3])

In [4]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [5]:
# 创建 Series 带有一个可以对各个数据点进行标记的索引
obj2 = Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [6]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

In [7]:
# 与普通的 NumPy 数组项目可通过索引方式选取单个或一组值
obj2['a']

-5

In [8]:
obj2['d'] = 6

In [9]:
obj2[ ['c', 'a', 'd'] ]

c    3
a   -5
d    6
dtype: int64

NumPy 数组运算(如根据布尔型数组进行过滤、标量乘法、应用数学函数等)都会保留索引和值之间的链接:

In [10]:
obj2

d    6
b    7
a   -5
c    3
dtype: int64

In [11]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [12]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [14]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

可以将 Series 看成是一个定长的有序字典，因为它是索引值到数据值的一个映射.

In [15]:
'b' in obj2

True

In [16]:
'e' in obj2

False

通过字典来创建 Series, 没有给定 index 的时候，默认使用字典的键，并根据键排列 :

In [17]:
sdata = {
    'Ohio': 35000,
    'Texas': 71000,
    'Oregon': 16000,
    'Utah': 5000
}
obj3 = Series(sdata)
obj3

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

In [18]:
# 指定索引
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

上面例子中 sdata 中跟 states 索引相匹配的那个 3 个值会被找出来并放到相应的位置上，但由于 'California' 所对应的 sdata 值找不到，所以其结果就是为 NaN, pandas 的 isnull 和 notnull 函数可以用于检测缺失数据：

In [19]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [20]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [21]:
# 同样有实例方法
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

Series 最重要的一个功能是: 它在算术运算中会自动对齐不同索引的数据。

In [23]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Series 对象本身及其索引都有一个 name 属性, 该属性跟 pandas 其他的关键功能关系非常密切:

In [24]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

In [25]:
# 就地修改 idnex 
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

## DataFrame

DataFrame 是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型(数值、字符串、布尔值等)。DataFrame 即有行索引也有列索引，它可以被看作由 Series 组成的字典(共用一个索引)

构建 DataFrame 的办法有很多，最常用的一种是直接传入一个由等长列表或NumPy数组组成的字典

In [27]:
data = {
    'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 
    'year': [2000, 2001, 2002, 2001, 2002],
    'pop': [1.5, 1.7, 3.6, 2.4, 2.9]
}
frame = DataFrame(data)
frame

,pop,state,year
0,1.5,Ohio,2000
1,1.7,Ohio,2001
2,3.6,Ohio,2002
3,2.4,Nevada,2001
4,2.9,Nevada,2002


如果指定了列序列，则 DataFrame 的列就会按照指定顺序进行排列

In [28]:
DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9


和 Series 一样，如果传入的列在数据中找不到，就会产生 NA 值

In [44]:
frame2 = DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                   index=['one', 'two', 'three', 'four', 'five'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN


In [30]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

通过类似字典标记的方式或属性的方式，可以将 DataFrame 的列获取为一个 Series

In [31]:
frame['state']

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
Name: state, dtype: object

In [32]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
Name: year, dtype: int64

Tip: 返回的 Series 拥有原 DataFrame 相同的索引，且 name 属性也已经被相应地设置好了。

In [35]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

列可以通过赋值的方式进行修改

In [45]:
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
